In [ ]:
%load_ext autoreload
%autoreload 2

import torch

torch.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from scripts.log_animation_usd import create_anim_from_usd

seq = create_anim_from_usd('/home/jorge/anim.usdc', n_frames=20)

100%|██████████| 20/20 [00:05<00:00,  3.78it/s]


In [2]:
from text3d2video.util import read_obj_with_uvs
from text3d2video.utilities.camera_placement import turntable_extrinsics


mesh, verts_uvs, faces_uvs = read_obj_with_uvs('data/meshes/lion.obj')
R, T = turntable_extrinsics()

/home/jorge/miniconda3/envs/thesis4/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:546: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


In [17]:
from text3d2video.rendering import render_rgb_uv_map
from text3d2video.utilities.video_comparison import display_vids
from text3d2video.utilities.video_util import pil_frames_to_clip

name = "rumba_zoom_side"
uvs = render_rgb_uv_map(seq.meshes, seq.cams, seq.verts_uvs, seq.faces_uvs)
vid = pil_frames_to_clip(uvs, fps=15)
display_vids([vid], title=name)

In [18]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact

new_anim = AnimationArtifact.create_empty_artifact(name)
new_anim.write_anim_seq(seq)
new_anim.log_standalone()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: romeu. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (/tmp/local_artifacts/animation/rumba_zoom_side)... Done. 0.1s
